In [ ]:
!pip install git+https://github.com/googleapis/python-aiplatform.git@mb-release

In [16]:
from google.cloud import aiplatform
PROJECT_ID=!gcloud config get-value project # returns default project id 
PROJECT_ID=PROJECT_ID[0]
REGION='europe-west4'
BUCKET = "gs://"+PROJECT_ID

In [17]:
VERSION='v1'
JOB_NAME="telco-churn-job-"+VERSION
MODEL_NAME="telco-churn-model-"+VERSION
DATASET_ID='5848592619210276864'

# BQ

In [15]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET, location=REGION)

ds = aiplatform.Dataset('4731418436645683200') # TODO: Change that to the created dataset id

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=JOB_NAME, 
    python_package_gcs_uri=BUCKET+'/telco-churn/dist/trainer-0.1.tar.gz', 
    python_module_name='trainer.task', 
    container_uri='eu.gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest', 
    model_serving_container_image_uri='eu.gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest'
)

job.run(
    ds, replica_count=1, model_display_name=MODEL_NAME,
    base_output_dir=BUCKET+"/telco-churn/assets-"+VERSION,
    bigquery_destination="bq://"+PROJECT_ID,
    machine_type='n1-standard-4'
)

m = job.get_model()

endpoint = m.deploy(machine_type='n1-standard-4')

endpoint.predict([['Male', 0, True, False, True, False, 8, 20.25, '158.35', 'No', 'No internet service', 'No internet service', 'No internet service', 'No internet service', 'No internet service', 'Month-to-month', 'Mailed check', 'No'], ['Female', 0, False, False, True, True, 2, 87.15, '183.75', 'Fiber optic', 'No', 'No', 'No', 'Yes', 'No', 'Month-to-month', 'Bank transfer (automatic)', 'No']])

INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://myfirstproject-226013/telco-churn/assets-v1 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/europe-west4/training/8040508631600332800?project=478111835512
INFO:google.cloud.aiplatform.training_jobs:Training projects/478111835512/locations/europe-west4/trainingPipelines/8040508631600332800 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:Training projects/478111835512/locations/europe-west4/trainingPipelines/8040508631600332800 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:Training projects/478111835512/locations/europe-west4/trainingPipelines/8040508631600332800 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:Training projects/478111835512/locations/europe-west4/trainingPipelines/8040508631600332800 current 

Prediction(predictions=[False, True], deployed_model_id='4457895128027103232')

# CSV

In [9]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET, location=REGION)

ds = aiplatform.Dataset('5848592619210276864') # TODO: Change that to the created dataset id

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=JOB_NAME, 
    python_package_gcs_uri=BUCKET+'/telco-churn/dist/trainer-0.1.tar.gz', 
    python_module_name='trainer.task', 
    container_uri='eu.gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest', 
    model_serving_container_image_uri='eu.gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest'
)

job.run(
    ds, replica_count=1, model_display_name=MODEL_NAME,
    base_output_dir=BUCKET+"/telco-churn/assets-"+VERSION,
    machine_type='n1-standard-4'
)

m = job.get_model()

endpoint = m.deploy(machine_type='n1-standard-4')

endpoint.predict([['Male', 0, True, False, True, False, 8, 20.25, '158.35', 'No', 'No internet service', 'No internet service', 'No internet service', 'No internet service', 'No internet service', 'Month-to-month', 'Mailed check', 'No'], ['Female', 0, False, False, True, True, 2, 87.15, '183.75', 'Fiber optic', 'No', 'No', 'No', 'Yes', 'No', 'Month-to-month', 'Bank transfer (automatic)', 'No']])

INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://myfirstproject-226013/telco-churn/assets-v1 


FailedPrecondition: 400 Please provide TrainingPipeline.input_data_config.bigquery_destination as the Dataset 4731418436645683200 source is BigQuery.

In [30]:
!pip install google-cloud-aiplatform

In [85]:
from google.cloud import aiplatform


def create_hyperparameter_tuning_job_python_package_sample(
    project: str,
    display_name: str,
    executor_image_uri: str,
    package_uri: str,
    python_module: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.JobServiceClient(client_options=client_options)

    # study_spec
    metric = {
        "metric_id": "f1score",
        "goal": aiplatform.gapic.StudySpec.MetricSpec.GoalType.MAXIMIZE,
    }
    
    
    parameters = [
    {
        "parameter_id": "model_param_kernel",
        "categorical_value_spec": {"values": ['linear', 'poly', 'rbf', 'sigmoid']},
        "scale_type": aiplatform.gapic.StudySpec.ParameterSpec.ScaleType.SCALE_TYPE_UNSPECIFIED,
        "conditional_parameter_specs": [
            
            {
               "parameter_spec": {
                    "parameter_id": "model_param_degree",
                    "integer_value_spec": {"min_value": 1, "max_value": 3},
                    "scale_type": aiplatform.gapic.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE
                },
                "parent_categorical_values": {"values": ['linear']}
            }
        ]
    },
    {
        "parameter_id": "model_param_C",
        "double_value_spec": {"min_value": 1, "max_value": 10},
        "scale_type": aiplatform.gapic.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
    }]

    # trial_job_spec
    machine_spec = {
        "machine_type": "n1-standard-4",
    }
    worker_pool_spec = {
        "machine_spec": machine_spec,
        "replica_count": 1,
        "python_package_spec": {
            "executor_image_uri": executor_image_uri,
            "package_uris": [package_uri],
            "python_module": python_module,
            "args": ['--model_dir=gs://myfirstproject-226013/telco-churn/assets-201203122247',
                    '--data_format=bigquery',
                    '--training_data_uri=bq://myfirstproject-226013.dataset_4731418436645683200_tables_2021_03_12T09_45_56_066Z.training',
                    '--test_data_uri=bq://myfirstproject-226013.dataset_4731418436645683200_tables_2021_03_12T09_45_56_066Z.test',
                    '--validation_data_uri=bq://myfirstproject-226013.dataset_4731418436645683200_tables_2021_03_12T09_45_56_066Z.validation'],
         
        },
    }

    # hyperparameter_tuning_job
    hyperparameter_tuning_job = {
        "display_name": display_name,
        "max_trial_count": 8,
        "parallel_trial_count": 2,
        "study_spec": {
            "metrics": [metric],
            "parameters": parameters,
            "algorithm": aiplatform.gapic.StudySpec.Algorithm.ALGORITHM_UNSPECIFIED,
        },
        "trial_job_spec": {"worker_pool_specs": [worker_pool_spec]},
    }
    parent = f"projects/{project}/locations/{location}"
    response = client.create_hyperparameter_tuning_job(
        parent=parent, hyperparameter_tuning_job=hyperparameter_tuning_job
    )
    print("response:", response)

In [86]:
create_hyperparameter_tuning_job_python_package_sample(
    project="myfirstproject-226013",
    display_name="training_job_custom_ht_v1",
    executor_image_uri="eu.gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest",
    package_uri="gs://myfirstproject-226013/telco-churn/dist/trainer-0.1.tar.gz",
    python_module="trainer.task",
    location="europe-west4",
    api_endpoint="europe-west4-aiplatform.googleapis.com"
)

response: name: "projects/478111835512/locations/europe-west4/hyperparameterTuningJobs/394451995487895552"
display_name: "training_job_custom_ht_v1"
study_spec {
  metrics {
    metric_id: "f1score"
    goal: MAXIMIZE
  }
  parameters {
    parameter_id: "model_param_kernel"
    categorical_value_spec {
      values: "linear"
      values: "poly"
      values: "rbf"
      values: "sigmoid"
    }
    conditional_parameter_specs {
      parameter_spec {
        parameter_id: "model_param_degree"
        integer_value_spec {
          min_value: 1
          max_value: 3
        }
        scale_type: UNIT_LINEAR_SCALE
      }
      parent_categorical_values {
        values: "linear"
      }
    }
  }
  parameters {
    parameter_id: "model_param_C"
    double_value_spec {
      min_value: 1.0
      max_value: 10.0
    }
    scale_type: UNIT_LINEAR_SCALE
  }
}
max_trial_count: 8
parallel_trial_count: 2
trial_job_spec {
  worker_pool_specs {
    machine_spec {
      machine_type: "n1-standa

In [83]:
from google.cloud import aiplatform


def create_custom_job_sample(
    project: str,
    display_name: str,
    executor_image_uri: str,
    package_uri: str,
    python_module: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.JobServiceClient(client_options=client_options)
    machine_spec = {
        "machine_type": "n1-standard-4",
    }
    worker_pool_spec = {
        "machine_spec": machine_spec,
        "replica_count": 1,
        "python_package_spec": {
            "executor_image_uri": executor_image_uri,
            "package_uris": [package_uri],
            "python_module": python_module,
            "args": [],
            "env": [
            {
              "name": "AIP_VALIDATION_DATA_URI",
              "value": "bq://myfirstproject-226013.dataset_4731418436645683200_tables_2021_03_12T09_45_56_066Z.validation"
            },
            {
              "name": "AIP_TEST_DATA_URI",
              "value": "bq://myfirstproject-226013.dataset_4731418436645683200_tables_2021_03_12T09_45_56_066Z.test"
            },
            {
              "name": "AIP_DATA_FORMAT",
              "value": "bigquery"
            },
            {
              "name": "AIP_TRAINING_DATA_URI",
              "value": "bq://myfirstproject-226013.dataset_4731418436645683200_tables_2021_03_12T09_45_56_066Z.training"
            },
            {
              "name": "AIP_MODEL_DIR",
              "value": "gs://myfirstproject-226013/telco-churn/assets-2012031216390000/model"
            }
          ]
        },
    }

    # hyperparameter_tuning_job
    custom_job = {
        "display_name": display_name,
        "job_spec": {"worker_pool_specs": [worker_pool_spec]},
    }
    
    print(custom_job)
    parent = f"projects/{project}/locations/{location}"
    response = client.create_custom_job(parent=parent, custom_job=custom_job)
    print("response:", response)
    
create_custom_job_sample(
    project="myfirstproject-226013",
    display_name="training_job_custom_ht_v1",
    executor_image_uri="eu.gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest",
    package_uri="gs://myfirstproject-226013/telco-churn/dist/trainer-0.1.tar.gz",
    python_module="trainer.task",
    location="europe-west4",
    api_endpoint="europe-west4-aiplatform.googleapis.com"
)

{'display_name': 'training_job_custom_ht_v1', 'job_spec': {'worker_pool_specs': [{'machine_spec': {'machine_type': 'n1-standard-4'}, 'replica_count': 1, 'python_package_spec': {'executor_image_uri': 'eu.gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest', 'package_uris': ['gs://myfirstproject-226013/telco-churn/dist/trainer-0.1.tar.gz'], 'python_module': 'trainer.task', 'args': [], 'env': [{'name': 'AIP_VALIDATION_DATA_URI', 'value': 'bq://myfirstproject-226013.dataset_4731418436645683200_tables_2021_03_12T09_45_56_066Z.validation'}, {'name': 'AIP_TEST_DATA_URI', 'value': 'bq://myfirstproject-226013.dataset_4731418436645683200_tables_2021_03_12T09_45_56_066Z.test'}, {'name': 'AIP_DATA_FORMAT', 'value': 'bigquery'}, {'name': 'AIP_TRAINING_DATA_URI', 'value': 'bq://myfirstproject-226013.dataset_4731418436645683200_tables_2021_03_12T09_45_56_066Z.training'}, {'name': 'AIP_MODEL_DIR', 'value': 'gs://myfirstproject-226013/telco-churn/assets-2012031216390000/model'}]}}]}}


ValueError: Protocol message PythonPackageSpec has no "env" field.

In [29]:
!pip freeze

adal @ file:///home/conda/feedstock_root/build_artifacts/adal_1611297630154/work
aiohttp @ file:///home/conda/feedstock_root/build_artifacts/aiohttp_1610358547752/work
ansiwrap==0.8.4
appdirs @ file:///home/conda/feedstock_root/build_artifacts/appdirs_1603108395799/work
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1610522577486/work
arrow @ file:///home/conda/feedstock_root/build_artifacts/arrow_1610483719724/work
asn1crypto @ file:///home/conda/feedstock_root/build_artifacts/asn1crypto_1595949944546/work
astropy @ file:///home/conda/feedstock_root/build_artifacts/astropy_1610312280831/work
async-generator==1.10
async-timeout==3.0.1
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1605083924122/work
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backports.functools-lru-cache==1.6.1
binaryornot==0.4.4
black @ file:///home/conda/feedstock_root/build_artifacts/black-recipe_1599478779128/work
bleach @ f